In [ ]:
%load_ext autoreload
%autoreload 2
%aimport utils

import pandas as pd
import numpy as np
import altair as alt
from altair_saver import save
from os.path import join
import datetime
import dateutil.parser

from constants import COLUMNS, DATA_AGGREGATE_TYPES
from utils import (
    read_combined_daily_counts_df, 
    read_combined_by_country_daily_counts_df,
    apply_theme, get_country_color_map,
    get_visualization_subtitle
)
from web import for_website

In [ ]:
CATEGORY = "category"
CATEGORY_OF_INTEREST = "new_positive_cases"

country_color_map = get_country_color_map()

min_date = datetime.datetime(2020, 1, 27) + datetime.timedelta(hours=1)
max_date = datetime.datetime(2020, 3, 30) + datetime.timedelta(hours=1)

# Countries have different ids in the JHU data than in the 4CE data
country_map = {
    "US": "USA"
}

In [ ]:
def preprocess_daily_df(df_dc):
    # Adapted from 02_daily_counts_altair.ipynb
    CATEGORY = "category"

    # Wide to long
    df_dc = pd.melt(df_dc, id_vars=[
        COLUMNS.SITE_ID, COLUMNS.DATE,
        COLUMNS.MASKED_UPPER_BOUND_NEW_POSITIVE_CASES,
        COLUMNS.MASKED_UPPER_BOUND_PATIENTS_IN_ICU,
        COLUMNS.MASKED_UPPER_BOUND_NEW_DEATHS,
        COLUMNS.UNMASKED_SITES_NEW_POSITIVE_CASES,
        COLUMNS.UNMASKED_SITES_PATIENTS_IN_ICU,
        COLUMNS.UNMASKED_SITES_NEW_DEATHS,
        COLUMNS.MASKED_SITES_NEW_POSITIVE_CASES,
        COLUMNS.MASKED_SITES_PATIENTS_IN_ICU,
        COLUMNS.MASKED_SITES_NEW_DEATHS
    ])
    df_dc = df_dc.rename(columns={"variable": CATEGORY, "value": COLUMNS.NUM_PATIENTS})

    # Leave only the 'upper' and 'under' values for the certain 'category' only
    for c in [COLUMNS.NEW_POSITIVE_CASES, COLUMNS.PATIENTS_IN_ICU, COLUMNS.NEW_DEATHS]:
        filter_c = df_dc[CATEGORY] == c
        df_dc.loc[filter_c, "upper"] = df_dc.loc[filter_c, COLUMNS.NUM_PATIENTS] + df_dc.loc[filter_c, "masked_upper_bound_" + c]
        df_dc.loc[filter_c, "under"] = df_dc.loc[filter_c, COLUMNS.NUM_PATIENTS]
        df_dc.loc[filter_c, COLUMNS.NUM_PATIENTS] = df_dc.loc[filter_c, COLUMNS.NUM_PATIENTS] + df_dc.loc[filter_c, "masked_upper_bound_" + c] / 2.0
        
        # Add num of sites
        df_dc.loc[filter_c, COLUMNS.NUM_SITES] = df_dc["unmasked_sites_" + c] + df_dc["masked_sites_" + c]

    # Drop unused columns
    df_dc = df_dc.drop(columns=[
        COLUMNS.MASKED_UPPER_BOUND_NEW_POSITIVE_CASES,
        COLUMNS.MASKED_UPPER_BOUND_PATIENTS_IN_ICU,
        COLUMNS.MASKED_UPPER_BOUND_NEW_DEATHS,
        COLUMNS.UNMASKED_SITES_NEW_POSITIVE_CASES,
        COLUMNS.UNMASKED_SITES_PATIENTS_IN_ICU,
        COLUMNS.UNMASKED_SITES_NEW_DEATHS,
        COLUMNS.MASKED_SITES_NEW_POSITIVE_CASES,
        COLUMNS.MASKED_SITES_PATIENTS_IN_ICU,
        COLUMNS.MASKED_SITES_NEW_DEATHS
    ])
    
    # Remove zero num_sites
    df_dc = df_dc[df_dc[COLUMNS.NUM_SITES] != 0]

    df_dc = df_dc.loc[df_dc["category"] == CATEGORY_OF_INTEREST]
    df_dc = df_dc.rename(columns={"siteid": "country", "num_patients": "count"})

    return df_dc


# DailyCounts-CombinedByCountry.csv
df_dc = pd.read_csv("https://ndownloader.figshare.com/files/22346625")

df_dc = preprocess_daily_df(df_dc)
df_dc.head()

In [ ]:
# We only need the JHU data for the countries that exist in the 4CE data.
unique_countries = df_dc["country"].unique().tolist()
unique_countries

In [ ]:
# Parse date strings into date objects.
def convert_date(date_str):
    try:
        return dateutil.parser.parse(date_str)
    except:
        return np.nan

In [ ]:
# Transform the JHU data.
jhu_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/dcd4181613f512a6f75249fc77b63286aebe7271/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
jhu_df = pd.read_csv(jhu_url)

jhu_df = jhu_df.rename(columns={"Country/Region": "country", "Province/State": "state"})
jhu_df = jhu_df.drop(columns=["Lat", "Long"])

jhu_df["country"] = jhu_df["country"].apply(lambda c: country_map[c] if c in country_map else c)
jhu_df = jhu_df.loc[jhu_df["country"].isin(unique_countries)]
jhu_df = jhu_df.loc[~pd.notna(jhu_df["state"])]
jhu_df = jhu_df.drop(columns=["state"])

jhu_df = jhu_df.melt(id_vars=["country"], var_name="date", value_name="cumulative_count")
jhu_df["date"] = jhu_df["date"].astype(str)
jhu_df["date"] = jhu_df["date"].apply(convert_date)
jhu_df = jhu_df.sort_values(by="date", ascending=True)
jhu_df = jhu_df.loc[(jhu_df["date"] >= min_date) & (jhu_df["date"] <= max_date)]

jhu_df_freeze = jhu_df.copy()

jhu_roc_df = pd.DataFrame(index=[], data=[], columns=["country", "date", "cumulative_count"])
for country, country_df in jhu_df.groupby("country"):
    country_df = country_df.copy()
    country_df["change"] = np.concatenate((np.array([np.nan]), np.diff(country_df["cumulative_count"].values)))
    country_df["cumulative_count"] = country_df["cumulative_count"].replace(0, np.nan)
    
    cumulative_count_max = country_df["cumulative_count"].max()
    
    country_df["change"] = country_df["change"] / cumulative_count_max

    jhu_roc_df = jhu_roc_df.append(country_df, ignore_index=True)
jhu_roc_df.head()

In [ ]:
# Transform the 4CE data to obtain normalized change values.
df_dc = df_dc.loc[df_dc["category"] == CATEGORY_OF_INTEREST]
df_dc["date"] = df_dc["date"].astype(str)
df_dc["date"] = df_dc["date"].apply(convert_date)
df_dc = df_dc.sort_values(by="date", ascending=True)
df_dc = df_dc.loc[(df_dc["date"] >= min_date) & (df_dc["date"] <= max_date)]

df_dc_freeze = df_dc.copy()

dc_roc_df = pd.DataFrame(index=[], data=[], columns=["country", "date", "count"])
for country, country_df in df_dc.groupby("country"):
    country_df = country_df.copy()
    country_df["cumulative_count"] = np.cumsum(country_df["count"].values)
    country_df["cumulative_count"] = country_df["cumulative_count"].replace(0, np.nan)
    
    country_df["cumulative_upper"] = np.cumsum(country_df["upper"].values)
    country_df["cumulative_upper"] = country_df["cumulative_upper"].replace(0, np.nan)
    
    country_df["cumulative_under"] = np.cumsum(country_df["under"].values)
    country_df["cumulative_under"] = country_df["cumulative_under"].replace(0, np.nan)
    
    
    cumulative_count_max = country_df["cumulative_count"].max()
    
    country_df["change"] = country_df["count"] / cumulative_count_max
    
    country_df["change_upper"] = country_df["upper"] / cumulative_count_max
    country_df["change_under"] = country_df["under"] / cumulative_count_max
    

    dc_roc_df = dc_roc_df.append(country_df, ignore_index=True)
dc_roc_df.head()

### Visualization of normalized change and country cumulative counts

In [ ]:
dc_roc_df = dc_roc_df.loc[(dc_roc_df["date"] >= min_date) & (dc_roc_df["date"] <= max_date)]
jhu_roc_df = jhu_roc_df.loc[(jhu_roc_df["date"] >= min_date) & (jhu_roc_df["date"] <= max_date)]

color_scale = alt.Scale(domain=list(country_color_map.keys()), range=list(country_color_map.values()))

country_selection = alt.selection_multi(fields=["country"], bind="legend")
country = alt.condition(country_selection, alt.Color("country:N", scale=color_scale, legend=alt.Legend(title="Country")), alt.value("#EAEAEA"))

date_domain = [alt.DateTime(year=min_date.year, month=min_date.month, date=min_date.day), alt.DateTime(year=max_date.year, month=max_date.month, date=max_date.day)]
date_scale = alt.X("date:T", scale=alt.Scale(domain=date_domain), title="Date")


pct_domain = [0.0, 0.25]
count_domain = [1, 1000000]

plot = (
    (
        (
            alt.Chart(dc_roc_df)
                .mark_line()
                .encode(
                    x=date_scale,
                    y=alt.Y("change:Q", scale=alt.Scale(domain=pct_domain), title="Normalized Change"),
                    color=country
                )
                .properties(title="Rate of Change per Country (4CE)")
            +
            alt.Chart(dc_roc_df)
                .mark_errorband()
                .encode(
                    x=date_scale,
                    y=alt.Y("change_upper:Q", scale=alt.Scale(domain=pct_domain), title="Normalized Change"), 
                    y2="change_under:Q",
                    color=country
                )
        ).resolve_scale(y="shared").interactive()
    | 
        (
            alt.Chart(dc_roc_df)
                .mark_line()
                .encode(
                    x=date_scale,
                    y=alt.Y("cumulative_count:Q", scale=alt.Scale(type="log", domain=count_domain), title="Cumulative Count"),
                    color=country
                )
                .properties(title="Country Cumulative Counts (4CE)")   
            +
            alt.Chart(dc_roc_df)
                .mark_errorband()
                .encode(
                    x=date_scale,
                    y=alt.Y("cumulative_upper:Q", scale=alt.Scale(type="log", domain=count_domain), title="Cumulative Count"), 
                    y2="cumulative_under:Q",
                    color=country
                )   
        ).resolve_scale(color="shared", y="shared", x="shared")
    ) & (
    alt.Chart(jhu_roc_df)
        .mark_line()
        .encode(
            x=date_scale,
            y=alt.Y("change:Q", scale=alt.Scale(domain=pct_domain), title="Normalized Change"),
            color=country
        )
        .properties(title="Rate of Change per Country (JHU)")
     | alt.Chart(jhu_roc_df)
        .mark_line()
        .encode(
            x=date_scale,
            y=alt.Y("cumulative_count:Q", scale=alt.Scale(type="log", domain=count_domain), title="Cumulative Count"),
            color=country
        )
        .properties(title="Country Cumulative Counts (JHU)")
    )
).add_selection(
    country_selection
)

plot

### Transform data for plots faceted by country

In [ ]:
jhu_roc_df = jhu_roc_df.copy()
dc_roc_df = dc_roc_df.copy()
jhu_roc_df["source"] = "JHU CSSE"
dc_roc_df["source"] = "4CE"

In [ ]:
jhu_roc_df.head()

In [ ]:
dc_roc_df.head()

In [ ]:
join_df = jhu_roc_df.append(dc_roc_df, ignore_index=True)
join_df["country_source"] = join_df.apply(lambda row: row["country"] + "_" + row["source"], axis='columns')
join_df.head()

### Visualization of normalized change faceted by country, with number of sites data

In [ ]:
title = "New Positive Cases, Change by Country, Comparison to JHU CSSE Data"

source_selection = alt.selection_multi(fields=["source"], bind="legend")

date_domain = [alt.DateTime(year=min_date.year, month=min_date.month, date=min_date.day), alt.DateTime(year=max_date.year, month=max_date.month, date=max_date.day)]

sites_domain = [0, dc_roc_df["num_sites"].max() + 1]
patients_domain = [0, dc_roc_df["count"].max() + 1]

country_names = list(country_color_map.keys())
country_source_names = [c + "_" + "4CE" for c in country_names] + [c + "_" + "JHU CSSE" for c in country_names]
color_scale = alt.Scale(domain=country_names, range=list(country_color_map.values()))
join_color_scale = alt.Scale(domain=country_source_names, range=list(country_color_map.values()) + ["#707070"]*len(country_names))

country_width = 170

nearest = alt.selection_single(encodings=['x', 'y'], on="mouseover", nearest=True, empty="none", clear="mouseout")
y_selection = alt.selection_interval(encodings=["y"], bind="scales")
date_brush = alt.selection(type='interval', encodings=['x'])

tooltip = [
    alt.Tooltip("source", title="Data source"),
    alt.Tooltip("country", title="Country"),
    alt.Tooltip("change", title="Normalized change", format=".3f"),
    alt.Tooltip("num_sites", title="Number of sites"),
    alt.Tooltip("count", title="Number of new cases"),
    alt.Tooltip("date", title="Date"),
]

rule = alt.Chart().mark_rule(color="red", size=0.5).encode(
    x="date:T"
).transform_filter(
    nearest
)

line = (
    alt.Chart(join_df)
        .transform_filter(source_selection)
        .transform_filter(date_brush)
        .mark_line(opacity=0.7)
        .encode(
            x=alt.X("date:T", title=None, axis=alt.Axis(labelBound=True), scale=alt.Scale(padding=5)),
            y=alt.Y("change:Q", axis=alt.Axis(title="Normalized change"), scale=alt.Scale(zero=False, nice=False, padding=5)),
            strokeDash=alt.StrokeDash("source:N", scale=alt.Scale(domain=["4CE", "JHU CSSE"], range=[[0,0], [3,3]]), 
            legend=alt.Legend(title="Data Source")),
            color=alt.Color("country_source:N", scale=join_color_scale, legend=None),
            tooltip=tooltip
        )
        .properties(width=country_width, height=200)
)
circle = (
    line.mark_circle()
        .encode(
            size=alt.condition(~nearest, alt.value(5), alt.value(30))
        )
        .add_selection(nearest)
)

num_sites_bar_bg = (
    alt.Chart(dc_roc_df)
        .mark_bar(size=2)
        .encode(
            x=alt.X("date:T", scale=alt.Scale(domain=date_domain, padding=5), title=None, axis=alt.Axis(labelBound=True)),
            y=alt.Y("num_sites:Q", axis=alt.Axis(title="# of sites"), scale=alt.Scale(domain=sites_domain)),
            color=alt.value("gray"),
            tooltip=tooltip
        )
        .properties(width=country_width, height=60) 
)
num_sites_bar = (
    num_sites_bar_bg
        .encode(
            color=alt.Color("country:N", scale=color_scale, legend=None),
        )
        .transform_filter(date_brush)
)

num_patients_bar_bg = (
    alt.Chart(dc_roc_df)
        .mark_bar(size=2)
        .encode(
            x=alt.X("date:T", scale=alt.Scale(domain=date_domain, padding=5), title=None, axis=alt.Axis(labelBound=True)),
            y=alt.Y("count:Q", axis=alt.Axis(title="# of new cases"), scale=alt.Scale(domain=patients_domain)),
            color=alt.value("gray"),
            tooltip=tooltip
        )
        .properties(width=country_width, height=60) 
)
num_patients_bar = (
    num_patients_bar_bg
        .encode(
            color=alt.Color("country:N", scale=color_scale, legend=None),
        )
        .transform_filter(date_brush)
)


top = (
    alt.layer(line, circle, rule, data=join_df)
        .facet(
            column=alt.Column("country:N"), bounds="flush" #header=alt.Header(labels=False)
        )
        .add_selection(y_selection)
)

num_sites_bottom = (
    alt.layer(num_sites_bar_bg, num_sites_bar, rule, data=dc_roc_df)
        .facet(
            column=alt.Column("country:N", header=alt.Header(labels=False)), bounds="flush"
        )
        .add_selection(nearest)
        .add_selection(date_brush)
)

num_patients_bottom = (
    alt.layer(num_patients_bar_bg, num_patients_bar, rule, data=dc_roc_df)
        .facet(
            column=alt.Column("country:N", header=alt.Header(labels=False)), bounds="flush"
        )
        .add_selection(nearest)
        .add_selection(date_brush)
)

plot = (
    alt.vconcat(top, num_patients_bottom, num_sites_bottom, spacing=5)
        .resolve_scale(color="shared", x="independent")
        .properties(title={
                "text": title, 
                "subtitle": get_visualization_subtitle(),
                "subtitleColor": "gray",
                "dx": 60
        })
        .add_selection(source_selection)
)

plot = apply_theme(
    plot, 
    axis_label_font_size=10, 
    axis_title_font_size=12, 
    axis_title_padding=8, 
    legend_orient="bottom", 
    legend_symbol_type="stroke",
    legend_title_orient="left",
    legend_title_font_size=14,
    label_font_size=12
).configure_header(title=None, labelPadding=3, labelFontSize=13)

for_website(plot, "Daily Count", "Normalized change by country")

plot

## Compute confidence intervals




For the CI:


- For change from previous day
    - $N1 = \texttt{count today}$
    - $N2 = \texttt{count yesterday}$
    - $R = N1/N2$
    - $C = R-1$
    - then the standard error of C is $\sqrt{(R+R^2)/N2}$
    - so the 95% CI for C is $C \pm 1.96*\sqrt{(R+R^2)/N2}$



In [ ]:
# Create a more clean dataframe, then compute the confidence intervals here and make the updated plot.
change_df = pd.DataFrame(columns=['source', 'country', 'date', 'date_str', 'N1', 'N2'])
for country in unique_countries:
    # Get the data frame for this country only.
    jhu_country_df = jhu_df_freeze.loc[jhu_df_freeze['country'] == country].copy()
    jhu_country_df["count"] = np.concatenate((np.array([np.nan]), np.diff(jhu_country_df["cumulative_count"].values)))
    site_country_df = df_dc_freeze.loc[df_dc_freeze['country'] == country].copy()
    # Transform dates into strings to do matching
    jhu_country_df['date_str'] = jhu_country_df['date'].astype(str)
    site_country_df['date_str'] = site_country_df['date'].astype(str)
    jhu_country_df = jhu_country_df.set_index('date_str')
    site_country_df = site_country_df.set_index('date_str')
    # Iterate over days
    jhu_count_yesterday = np.nan
    site_count_yesterday = np.nan
    for date in [min_date + datetime.timedelta(days=x) for x in range((max_date - min_date).days)]:
        date_str = str(date.date())
        
        try:
            jhu_count_today = jhu_country_df.at[date_str,"count"]
        except KeyError:
            jhu_count_today = np.nan
        try:
            site_count_today = site_country_df.at[date_str, "count"]
        except KeyError:
            site_count_today = np.nan
        
        try:
            num_sites = site_country_df.at[date_str, "num_sites"]
        except KeyError:
            num_sites = np.nan
        
        # Append values to the dataframe.
        change_df = change_df.append({
            'source': 'JHU CSSE',
            'country': country,
            'date': date,
            'date_str': date_str,
            'N1': jhu_count_today,
            'N2': jhu_count_yesterday,
            'num_sites': np.nan
        }, ignore_index=True)
        change_df = change_df.append({
            'source': '4CE',
            'country': country,
            'date': date,
            'date_str': date_str,
            'N1': site_count_today,
            'N2': site_count_yesterday,
            'num_sites': num_sites
        }, ignore_index=True)
        
        jhu_count_yesterday = jhu_count_today
        site_count_yesterday = site_count_today

In [ ]:
change_df = change_df.fillna(0) # TODO: figure out how to handle NaN values in each step
change_df['R'] = change_df['N1'] / change_df['N2']
change_df['C'] = change_df['R'] - 1 # TODO: did R-1 mean R_-1 (R "sub 1", the R from the previous day)?
change_df['standard_error'] = change_df.apply(lambda obs: (obs['R']+np.power(obs['R'], 2))/obs['N2'], axis='columns')
change_df['95_CI_below'] = change_df.apply(lambda obs: obs['C'] - 1.96*np.sqrt(obs['standard_error']), axis='columns')
change_df['95_CI_above'] = change_df.apply(lambda obs: obs['C'] + 1.96*np.sqrt(obs['standard_error']), axis='columns')
change_df = change_df.replace([np.inf, -np.inf], np.nan)
change_df["country_source"] = change_df.apply(lambda row: row["country"] + "_" + row["source"], axis='columns')
change_df['date'] = change_df['date_str']
change_df = change_df.fillna(0)
change_df.head()

In [ ]:
title = "New Positive Cases, Change by Country, Comparison to JHU CSSE Data"

change_fce_df = change_df.loc[change_df['source'] == '4CE'].copy()

source_selection = alt.selection_multi(fields=["source"], bind="legend")

date_domain = [alt.DateTime(year=min_date.year, month=min_date.month, date=min_date.day), alt.DateTime(year=max_date.year, month=max_date.month, date=max_date.day)]

sites_domain = [0, change_fce_df["num_sites"].max() + 1]
patients_domain = [0, change_fce_df["N1"].max() + 1]

country_names = list(country_color_map.keys())
country_source_names = [c + "_" + "4CE" for c in country_names] + [c + "_" + "JHU CSSE" for c in country_names]
color_scale = alt.Scale(domain=country_names, range=list(country_color_map.values()))
join_color_scale = alt.Scale(domain=country_source_names, range=list(country_color_map.values()) + ["#707070"]*len(country_names))

country_width = 170

nearest = alt.selection_single(encodings=['x', 'y'], on="mouseover", nearest=True, empty="none", clear="mouseout")
y_selection = alt.selection_interval(encodings=["y"], bind="scales")
date_brush = alt.selection(type='interval', encodings=['x'])

tooltip = [
    alt.Tooltip("source", title="Data source"),
    alt.Tooltip("country", title="Country"),
    alt.Tooltip("num_sites", title="Number of sites"),
    alt.Tooltip("N1", title="Number of new cases"),
    alt.Tooltip("date", title="Date"),
    alt.Tooltip("95_CI_below", title="95% CI upper bound"),
    alt.Tooltip("C", title="Normalized change", format=".3f"),
    alt.Tooltip("95_CI_above", title="95% CI lower bound")
]

rule = alt.Chart().mark_rule(color="red", size=0.5).encode(
    x="date:T"
).transform_filter(
    nearest
)

line = (
    alt.Chart(change_df)
        .transform_filter(source_selection)
        .transform_filter(date_brush)
        .mark_line(opacity=0.7)
        .encode(
            x=alt.X("date:T", title=None, axis=alt.Axis(labelBound=True), scale=alt.Scale(padding=5)),
            y=alt.Y("C:Q", axis=alt.Axis(title="Normalized change"), scale=alt.Scale(zero=False, nice=False, padding=5)),
            strokeDash=alt.StrokeDash("source:N", scale=alt.Scale(domain=["4CE", "JHU CSSE"], range=[[0,0], [3,3]]), 
            legend=alt.Legend(title="Data Source")),
            color=alt.Color("country_source:N", scale=join_color_scale, legend=None),
            tooltip=tooltip
        )
        .properties(width=country_width, height=200)
)

errorband = (
    line.mark_errorband().encode(
        x=alt.X(f"date:T", axis=alt.Axis(labelBound=True), title=None),
        y=alt.Y(f"95_CI_above:Q", title=""), 
        y2=alt.Y2(f"95_CI_below:Q", title=""),
        color=alt.Color(f"country_source:N", scale=color_scale, legend=alt.Legend(title=None)),
        tooltip=tooltip
    )
)

circle = (
    line.mark_circle()
        .encode(
            size=alt.condition(~nearest, alt.value(5), alt.value(30))
        )
        .add_selection(nearest)
)

num_sites_bar_bg = (
    alt.Chart(change_fce_df)
        .mark_bar(size=2)
        .encode(
            x=alt.X("date:T", scale=alt.Scale(domain=date_domain, padding=5), title=None, axis=alt.Axis(labelBound=True)),
            y=alt.Y("num_sites:Q", axis=alt.Axis(title="# of sites"), scale=alt.Scale(domain=sites_domain)),
            color=alt.value("gray"),
            tooltip=tooltip
        )
        .properties(width=country_width, height=60) 
)

num_sites_bar = (
    num_sites_bar_bg
        .encode(
            color=alt.Color("country:N", scale=color_scale, legend=None),
        )
        .transform_filter(date_brush)
)

num_patients_bar_bg = (
    alt.Chart(change_fce_df)
        .mark_bar(size=2)
        .encode(
            x=alt.X("date:T", scale=alt.Scale(domain=date_domain, padding=5), title=None, axis=alt.Axis(labelBound=True)),
            y=alt.Y("N1:Q", axis=alt.Axis(title="# of new cases"), scale=alt.Scale(domain=patients_domain)),
            color=alt.value("gray"),
            tooltip=tooltip
        )
        .properties(width=country_width, height=60) 
)

num_patients_bar = (
    num_patients_bar_bg
        .encode(
            color=alt.Color("country:N", scale=color_scale, legend=None),
        )
        .transform_filter(date_brush)
)

top = (
    alt.layer(line, errorband, circle, rule, data=change_df)
        .facet(
            column=alt.Column("country:N"), bounds="flush" #header=alt.Header(labels=False)
        )
        .add_selection(y_selection)
)

num_sites_bottom = (
    alt.layer(num_sites_bar_bg, num_sites_bar, rule, data=change_fce_df)
        .facet(
            column=alt.Column("country:N", header=alt.Header(labels=False)), bounds="flush"
        )
        .add_selection(nearest)
        .add_selection(date_brush)
)

num_patients_bottom = (
    alt.layer(num_patients_bar_bg, num_patients_bar, rule, data=change_fce_df)
        .facet(
            column=alt.Column("country:N", header=alt.Header(labels=False)), bounds="flush"
        )
        .add_selection(nearest)
        .add_selection(date_brush)
)

plot = (
    alt.vconcat(top, num_patients_bottom, num_sites_bottom, spacing=5)
        .resolve_scale(color="shared", x="independent")
        .properties(title={
                "text": title, 
                "subtitle": get_visualization_subtitle(),
                "subtitleColor": "gray",
                "dx": 60
        })
        .add_selection(source_selection)
)

plot = apply_theme(
    plot, 
    axis_label_font_size=10, 
    axis_title_font_size=12, 
    axis_title_padding=8, 
    legend_orient="bottom", 
    legend_symbol_type="stroke",
    legend_title_orient="left",
    legend_title_font_size=14,
    label_font_size=12
).configure_header(title=None, labelPadding=3, labelFontSize=13)

for_website(plot, "Daily Count", "Normalized change by country with confidence intervals")

plot

In [ ]:
#change_df.to_csv("change_confidence_intervals.csv")

## Normalized New Daily Cases

First, obtain total hospital discharges for each country.

$\texttt{country_total} = \text{country total in-patient-discharge}$

$\texttt{country_4CE_total} = \text{total in-patient-discharge in our sites within that country}$

$F0 = \frac{\texttt{country_total}}{\texttt{country_4CE_total}}$

$F0$ is used to normalize.

- For new figure that shows daily case # per 100K, we will instead show 
    - $\texttt{RATE} = \texttt{N_case} * F1$
    - where $F1 = F0 * \frac{100K}{\texttt{country population}}$
    - then the standard error for $\texttt{RATE}$ will be $\sqrt(\texttt{RATE}*F1)$ and the confidence interval will be $\texttt{RATE} \pm 1.96*\sqrt(\texttt{RATE}*F1)$

In [ ]:
# Get daily new cases from cumulative_count
norm_jhu = jhu_roc_df.copy()

norm_jhu = norm_jhu[norm_jhu["country"] != "Singapore"].reset_index() # Remove Singapore

norm_jhu["count"] = norm_jhu["cumulative_count"].diff()

norm_jhu.loc[norm_jhu["date"] == "2020-01-28", "count"] = np.nan # Make sure the start count is NaN

norm_jhu

In [ ]:
norm_4ce = dc_roc_df.copy()

norm_4ce = norm_4ce[norm_4ce["country"] != "Singapore"].reset_index() # Remove Singapore

norm_4ce.head()

In [ ]:
population = {
    # Adopted from https://www.worldometers.info on May 12, 2020
    "France": 65254341,
    "USA": 330743891,
    "Germany": 83748343,
    "Italy": 60473574
}
F0 = {
    "France": 8.06,
    "USA": 47.68,
    "Germany": 153.6,
    "Italy": 70.9
}
norm_jhu["population"] = norm_jhu["country"]
norm_jhu["population"] = norm_jhu["population"].apply(lambda x: population[x])
# norm_jhu["F0"] = norm_jhu["country"]
# norm_jhu["F0"] = norm_jhu["F0"].apply(lambda x: F0[x])

norm_4ce["population"] = norm_4ce["country"]
norm_4ce["population"] = norm_4ce["population"].apply(lambda x: population[x])
norm_4ce["F0"] = norm_4ce["country"]
norm_4ce["F0"] = norm_4ce["F0"].apply(lambda x: F0[x])

norm_4ce.head()

In [ ]:
norm_jhu["adjusted_count"] = norm_jhu["count"]
norm_jhu["F1"] = 100000 / norm_jhu["population"]
norm_jhu["RATE"] = norm_jhu["count"] * norm_jhu["F1"]
# norm_jhu["std_error"] = norm_jhu["F1"] * norm_jhu["RATE"]
# norm_jhu["std_error"] = norm_jhu["std_error"].apply(lambda x: np.sqrt(x))
# norm_jhu["ci_above"] = 0 # norm_jhu["RATE"] + 1.96 * norm_jhu["std_error"]
# norm_jhu["ci_below"] = 0 # norm_jhu["RATE"] - 1.96 * norm_jhu["std_error"]

norm_4ce["adjusted_count"] = norm_4ce["F0"] * norm_4ce["count"]
norm_4ce["F1"] = norm_4ce["F0"] * 100000 / norm_4ce["population"]
norm_4ce["RATE"] = norm_4ce["count"] * norm_4ce["F1"]
norm_4ce["std_error"] = norm_4ce["F1"] * norm_4ce["RATE"]
norm_4ce["std_error"] = norm_4ce["std_error"].apply(lambda x: np.sqrt(x))
norm_4ce["ci_above"] = norm_4ce["RATE"] + 1.96 * norm_4ce["std_error"]
norm_4ce["ci_below"] = norm_4ce["RATE"] - 1.96 * norm_4ce["std_error"]

norm_jhu_min_col = norm_jhu[['country', 'date', 'count', 'adjusted_count', 'source', 'population', 'F0', 'F1', 'RATE', 'std_error', 'ci_above', 'ci_below']]
norm_4ce_min_col = norm_4ce[['country', 'date', 'count', 'adjusted_count', 'source', 'population', 'F0', 'F1', 'RATE', 'std_error', 'ci_above', 'ci_below']]

norm_jhu_min_col.head()
norm_4ce_min_col.head()

In [ ]:
norm_df = norm_jhu_min_col.append(norm_4ce_min_col)

norm_df["country_source"] = norm_df.apply(lambda row: row["country"] + "_" + row["source"], axis='columns')

norm_df.to_csv("norm_df.csv")

norm_df

In [ ]:
title = "Country-Level Positive Case Rate, Comparison to JHU CSSE Data"

# Selection
source_selection = alt.selection_multi(fields=["source"], bind="legend")

# Domains
date_domain = [alt.DateTime(year=min_date.year, month=min_date.month, date=min_date.day), alt.DateTime(year=max_date.year, month=max_date.month, date=max_date.day)]

country_names = ["France", "Germany", "Italy", "USA"]
COUNTRY_COLORS = ["#0072B2", "#E69F00", "#009E73", "#D55E00"]
country_source_names = [c + "_" + "4CE" for c in country_names] + [c + "_" + "JHU CSSE" for c in country_names]
color_scale = alt.Scale(domain=country_names, range=COUNTRY_COLORS)
join_color_scale = alt.Scale(domain=country_source_names, range=COUNTRY_COLORS + ["#707070"] * len(country_names))

country_width = 170

nearest = alt.selection_single(encodings=['x', 'y'], on="mouseover", nearest=True, empty="none", clear="mouseout")
y_selection = alt.selection_interval(encodings=["y"], bind="scales")

# Additional Visual Elements
tooltip = [
    alt.Tooltip("source", title="Data source"),
    alt.Tooltip("country", title="Country"),
    alt.Tooltip("count", title="Normalized Daily Cases"),
    alt.Tooltip("date", title="Date"),
    alt.Tooltip("ci_below", title="95% CI upper bound"),
    alt.Tooltip("ci_above", title="95% CI lower bound")
]

rule = alt.Chart().mark_rule(color="red", size=0.5).encode(
    x="date:T"
).transform_filter(
    nearest
)

line = alt.Chart(norm_df).transform_filter(source_selection).mark_line(opacity=0.7).encode(
    x=alt.X("date:T", title=None, axis=alt.Axis(labelBound=True), scale=alt.Scale(padding=5)),
    y=alt.Y("RATE:Q", axis=alt.Axis(title="Country-level case rate"), scale=alt.Scale(zero=False, nice=False, padding=5)),
    strokeDash=alt.StrokeDash("source:N", scale=alt.Scale(domain=["4CE", "JHU CSSE"], range=[[0,0], [3,3]]), 
    legend=alt.Legend(title="Data Source")),
    color=alt.Color("country_source:N", scale=join_color_scale, legend=None),
    tooltip=tooltip
).properties(width=country_width, height=200)

errorband = line.transform_filter(alt.datum["source"] == "4CE").mark_errorband().encode(
    x=alt.X(f"date:T", title=None, axis=alt.Axis(labelBound=True)),
    y=alt.Y(f"sum(ci_below):Q", title=""),
    y2=alt.Y2(f"sum(ci_above):Q", title=""),
    color=alt.Color(f"country:N", scale=color_scale, legend=alt.Legend(title=None)),
    tooltip=tooltip
)

circle = (
    line.mark_circle()
        .encode(
            size=alt.condition(~nearest, alt.value(5), alt.value(30))
        )
        .add_selection(nearest)
)

top = alt.layer(line, errorband, circle, rule, data=norm_df).facet(
# top = alt.layer(line, errorband, circle, data=norm_df).facet(
    column=alt.Column("country:N"), bounds="flush"
).add_selection(y_selection).add_selection(source_selection).properties(
    title={
        "text": title, 
        "subtitle": get_visualization_subtitle(),
        "subtitleColor": "gray",
        "dx": 60
})

plot = apply_theme(
    top, 
    axis_label_font_size=10, 
    axis_title_font_size=12, 
    axis_title_padding=8, 
    legend_orient="bottom", 
    legend_symbol_type="stroke",
    legend_title_orient="left",
    legend_title_font_size=14,
    label_font_size=12
).configure_header(title=None, labelPadding=3, labelFontSize=13)

for_website(plot, "Daily Count", "country-level rate of positive cases")

plot